# **Tools Recognition**

In [ ]:
# Istallazione pacchetti se necessario.

# pip install opencv-python

In [1]:
# pacchetti utilizzati.
import os
import cv2
import glob
import numpy as np
from PIL import Image
from tqdm import tqdm

### 1) Raccolta e etichettatura delle immagini

- 10 diverse categorie:
    - accendino
    - cacciavite
    - chiave
    - forbici
    - martello
    - metro
    - nastro
    - pappagallo
    - penna
    - spillatrice 

- 4 diverse superfici:
    - tavolo bianco
    - banco da lavoro
    - pavimento di cemento
    - coperta a righe
- 20 immagini per il training per ogni categoria
- 26 immagini per il test contenenti oggetti multipli e anche estranei alle 10 categorie


In [ ]:
# Oggetti utilizzati.
tool_names = ['accendino', 'cacciavite', 'chiave', 'forbici', 'martello', 'metro', 'nastro', 'pappagallo', 'penna', 'spillatrice']

In [ ]:
# Rinomiamo le immagini nelle cartelle.
cwd = os.getcwd()
for tool in tool_names:
    path = os.path.join(cwd, f"images/{tool}/")
    image_files = glob.glob(path + '*.png')

    for i, file in enumerate(image_files):
        new_name = tool+ '_' + str(i+1) + '.png'
        os.rename(file, os.path.join(path, new_name))


# Rinominiamo le immagini del test set.
path = os.path.join(cwd, f"images/Test_set/")
image_files = glob.glob(path + '*.png')

for i, file in enumerate(image_files):
    new_name = 'test_' + str(i+1) + '.png'
    os.rename(file, os.path.join(path, new_name))

### 2) Pre-elaborazione delle immagini

- Ridimensionamento : portartiamo nello stesso formato tutte le immagini (520 x 520). Alcune delle utilità di questo passaggio possono essere una maggiore uniformità una volta che si va ad allenare l'algoritmo di calssificazione e l'aumento della diversità dei dati di training che possono portare il modello ad avere una maggiore robustezza;

In [ ]:
# Portiamo tutte le immagini nella stessa dimensione (520 x 520).
cwd = os.getcwd()

for tool in tool_names:
    path = os.path.join(cwd, f"images/{tool}/")
    image_files = glob.glob(path + '*.png')

    for i, file in enumerate(image_files):
        image = Image.open(file)

        # Ridimensionamento dell'immagine.
        resized_image = image.resize((520, 520))
        resized_image.save(file)

# test-set
path = os.path.join(cwd, f"images/Test_set/")
image_files = glob.glob(path + '*.png')

for i, file in enumerate(image_files):
    image = Image.open(file)

    # Ridimensionamento dell'immagine.
    resized_image = image.resize((520, 520))
    resized_image.save(file)

- Ricolorazione: applichiamo la scalda di grigi riducendo le immagini ad un unico canale rendendole più semplici da analizzare;
- Gaussian Filter: applichiamo un `Gaussian filter` con dimensione 5x5 per sfocare l'immagine e rendere lo sfondo piu semplice da distinguere rispetto agli oggetti;

### 3) Segmentazione delle immagini 

Tecniche di segmentazione prese dalle slide:
- per contorni:
    - Canny edge detector

- per regioni:
    - binarizzazione
    - sogliatura automatica OTSU

- mediante clustering:
    - k-mean


Problemi:
- ombre
- sfondo

In [ ]:
t = 'spillatrice'
for i in range(1,21):
    path = f'c:\\Users\\mcm23\\OneDrive\\Desktop\\GitHub\\Object_Detection\\images/{t}/{t}_{i}.1.png'
    path = f'c:\\Users\\mcm23\\OneDrive\\Desktop\\GitHub\\Object_Detection\\images/test_set/test_{i}.png'

    ## Canny Edge Detector
    # leggi l'immagine in scala di grigi
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    # Definizione del kernel di apertura
    kernel = np.ones((5,5), np.uint8)

    # Applicazione dell'operatore di apertura
    opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)

    # visualizza l'immagine con i bordi individuatia
    cv2.imshow('CANNY Edge Detector', opening)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


### PIPELINE

1) ``Raccolta dei dati``: Creiamo il nostro dataset di train e test etichettando le varie immagini e considerando varie situazioni: diverse posizioni degli oggetti, cambi di luminosità, presenza di ombre e diversi sfondi, alcuni dei quali presentano delle irregolarità.

2) ``Pre-elaborazione dell'immagine``: Prima della segmentazione creiamo un *ground truth* per ogni immagine in modo tale da poter valutare lo step successivo in temini di accuratezza e modifichiamo le immagini per preparale per i prossimi step: ridimensioniamo le immagini riducendole tutte alla stessa dimensione (520 x 520 pixel) e le ricoloriamo in scala di grigi per ridurle ad un singolo canale e renderle più pratiche e semplici da elaborare.

3) ``Segmentazione dell'immagine``: Una volta ottenuto l'immagine preparata, si applica un metodo di segmentazione per dividere l'immagine in regioni omogenee. Abbiamo provato ad utilizzare tecniche di segmentazione come la segmentazione basata sulla soglia, la segmentazione basata sulla regione o la segmentazione basata sui cluster: queste tecniche non hanno portato buoni risultati perchè, anche se identificano molto bene gli oggetti con una precisione quasi sempre superiore al 90%, con altrettanta precisione sbagliano ad identificare lo sfondo a causa della presenza di ombre e di sfondi che non permetono una chiara distinzione con l'oggetto considerato e presentano delle imperfezioni. Cerchiamo quindi un'altra tecnica...

---
 


4) ``Estrazione delle features``: Per ogni regione dell'immagine segmentata, si estraggono le feature utilizzate per addestrare il modello di classificazione. Le feature possono includere la forma, la texture, il colore o qualsiasi altra caratteristica che aiuti a distinguere le diverse classi di oggetti.

5) ``Addestramento del modello di classificazione``: Una volta estratte le feature, si addestra il modello di classificazione utilizzando un algoritmo di machine learning come le reti neurali, le SVM o gli alberi di decisione. Il modello viene addestrato per riconoscere le diverse classi di oggetti.

6) ``Rilevamento degli oggetti``: Una volta addestrato il modello di classificazione, si può applicare il modello all'immagine originale per rilevare gli oggetti di interesse. Utilizzando le feature estratte, il modello cerca le regioni dell'immagine che corrispondono agli oggetti di interesse.

7) ``Post-elaborazione dell'immagine``: Una volta rilevati gli oggetti, si possono applicare tecniche di post-elaborazione come la non-maximum suppression per rimuovere i falsi positivi e migliorare la precisione del modello.
